<a href="https://colab.research.google.com/github/jrakhshanda55/Ausaf_ML/blob/main/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import string
import re
import string
from datetime import datetime
import matplotlib.pyplot as plt
import random
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)

In [2]:
#!pip install allennlp

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [20]:
# from google.colab import drive
# drive.mount('/content/drive')

# Arxiv Data Preprocessing



In [6]:
#train = pd.read_csv('/content/train.zip',compression='zip')
data = pd.read_csv('/content/val.zip',compression='zip')

In [41]:
df = data.sample(n=15000, random_state=42)

In [42]:
df.category.value_counts()

astro-ph    1009
hep-ph       952
quant-ph     778
hep-th       632
cs.CV        632
            ... 
q-fin.TR       2
cs.NA          2
nlin.CG        2
q-fin.RM       2
cs.GL          2
Name: category, Length: 149, dtype: int64

## Training Context Aware Model

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from allennlp.modules.elmo import Elmo, batch_to_ids
import numpy as np


In [ ]:
#Download the ELMo model

#!wget https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json
#!wget https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5

In [36]:
# Load the ELMo options and weights files
options_file = "/content/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weights_file = "/content/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assuming 'corpus' contains your text data, and 'categories' contains the labels
X = df['corpus']
y = df['category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

# Encode the category column using LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Now, initialize and fine-tune ELMo embeddings (replace options_file and weights_file with your paths)
elmo = Elmo(options_file, weights_file, num_output_representations=1, dropout=0)

# Tokenize and convert to character IDs
X_train_ids = batch_to_ids(X_train)
X_test_ids = batch_to_ids(X_test)

# Get ELMo embeddings for training data
with torch.no_grad():
    elmo_train_embeddings = elmo(X_train_ids)["elmo_representations"][0]

# Get ELMo embeddings for testing data
with torch.no_grad():
    elmo_test_embeddings = elmo(X_test_ids)["elmo_representations"][0]

In [ ]:
# Perform your classification using the embeddings and encoded labels
# (You can use any classification model, e.g., a neural network or a traditional classifier)

# For example, using a simple logistic regression classifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Reshape the embeddings for the classifier
X_train_features = elmo_train_embeddings.mean(dim=1).numpy()
X_test_features = elmo_test_embeddings.mean(dim=1).numpy()

# Train a classifier
classifier = LogisticRegression(random_state=42)
classifier.fit(X_train_features, y_train_encoded)

# Make predictions on the test set
predictions = classifier.predict(X_test_features)

# Evaluate the accuracy
accuracy = accuracy_score(y_test_encoded, predictions)
print(f"Accuracy: {accuracy:.4f}")
